# Pattern Matrix Replacement

This this notebook we explore strategies to replace the ***Hit Correlation*** step of Konrad's pipeline, specifically the *patten matrix* algorithm previously used.

## Approach
I am thinking of two approaches. First, use a simple MLP to determine if two given points are *causally related* or not. To do this, the following tasks are required:
- [ ] prepare the data such that each row contains the features (x,y,z,time/timeslice) of each pair of points (exluding self)
    - [ ] subtask here is to use PCA to reduce the dimensions and see if the network performs same/worse
- [ ] create labels for training set ie. *1* of related and *0* otherwise (this hinges upon the fact that we can extract labels from *mc_info* table)
- [ ] visualize the results

The other approach is to treat this as an unsupervised learning task and use clustering to determine *related* points.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
data = pd.read_csv("../data/data.csv")

In [3]:
data

,dom_id,pmt_id,pos_x,pos_y,pos_z,dir_x,dir_y,dir_z,tot,time,label
0,321.0,5.0,-17.661,32.245,65.231,-0.460,-0.266,-0.847,26.0,0.0,0
1,1653.0,23.0,11.595,85.465,65.459,-0.955,-0.000,0.296,27.0,0.0,0
2,275.0,9.0,-36.464,67.166,160.189,0.415,0.720,-0.556,26.0,0.0,0
3,1660.0,23.0,61.660,101.635,169.059,-0.955,-0.000,0.296,26.0,0.0,0
4,966.0,16.0,-54.510,-78.323,94.341,-0.827,0.478,-0.296,24.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
45820211,523.0,13.0,-57.230,-5.401,196.389,0.415,-0.720,-0.556,28.0,101502104.0,0
45820212,387.0,12.0,0.724,66.341,121.789,-0.415,-0.720,-0.556,24.0,101516467.0,0
45820213,95.0,4.0,-26.436,86.737,160.131,-0.460,0.266,-0.847,23.0,101545421.0,0
45820214,741.0,30.0,-26.931,-21.994,178.511,0.000,-0.830,0.558,27.0,101581891.0,0


In [4]:
mc_info = pd.read_hdf("../data/events.h5", key="/data/mc_info")

/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
mc_info

,nu_E,type,nu_dir.x,nu.dir.y,nu.dir.z,nu.pos.x,nu.pos.y,nu.pos.z,nu.hits.start,nu.hits.end
0,15.540,-14,-0.630831,0.436518,0.641486,-85.337,154.304,35.735,0,5
1,11.458,14,0.070157,0.942066,-0.328009,19.081,169.073,-92.435,5,6
2,11.485,14,0.367514,0.049922,-0.928677,-61.314,130.036,-64.078,6,11
3,11.571,14,-0.177418,0.344584,0.921838,48.457,76.398,112.710,11,15
4,13.148,-14,-0.997387,0.065683,0.030071,138.488,-166.544,52.433,15,17
...,...,...,...,...,...,...,...,...,...,...
5730,73.038,-14,-0.119695,-0.113570,-0.986294,-102.301,10.814,259.625,489453,489455
5731,79.484,14,-0.220923,0.896138,-0.384876,51.252,56.680,72.604,489455,489817
5732,35.602,14,-0.665612,0.267922,0.696548,191.109,-60.389,-7.956,489817,489820
5733,10.022,14,0.868500,-0.195664,0.455438,-126.753,61.647,-71.618,489820,489897


In [6]:
mc_info.type.unique()

array([-14,  14])